In [22]:
import os
import psycopg
from datetime import date
from googleapiclient.discovery import build
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

# Collecter et enregistrer les métriques des chaînes

- On va collecter et enregistrer en même temps les métriques de chaque chaîne pertinente (en évitant de passer par un JSON).


In [4]:
youtube = build('youtube', 'v3', developerKey=os.getenv("YOUTUBE_API_KEY1"))

In [5]:
def getMetrics(id_chaine):
    request = youtube.channels().list(
        part='statistics',
        id=id_chaine
    )
    response = request.execute()
    
    if not response['items']:
        return None
    
    stats = response['items'][0]['statistics']
    date_releve_chaine = date.today().isoformat()
    nombre_vues_total = int(stats.get('viewCount', 0))
    nombre_abonnes_total = int(stats.get('subscriberCount', 0))
    nombre_videos_total = int(stats.get('videoCount', 0))
    
    return {
        'date_releve_chaine': date_releve_chaine,
        'nombre_vues_total': nombre_vues_total,
        'nombre_abonnes_total': nombre_abonnes_total,
        'nombre_videos_total': nombre_videos_total
    }

In [6]:
getMetrics("UCVQeGg4Fdrrr8vDXa7yjOYg")

{'date_releve_chaine': '2025-05-19',
 'nombre_vues_total': 18189595,
 'nombre_abonnes_total': 127000,
 'nombre_videos_total': 543}

In [10]:
conn = psycopg.connect(
    dbname="mydatabase",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Récupérer les chaînes pertinentes
cur.execute("SELECT id_chaine FROM chaines WHERE pertinente = TRUE")
chaines = cur.fetchall()


print(chaines)
print(len(chaines))

[('UCVQeGg4Fdrrr8vDXa7yjOYg',), ('UCbXqpYCVTjdz2dx1nRx6W_A',), ('UCmx-LdHmmK5uoeCxLCro0lA',), ('UCbLlJlku_L6ghxMd4BqBpqg',), ('UCPCydRQqXBQ91WXbJE0m_Nw',), ('UCfJJcIcDpbUc9BFl9pv2OBQ',), ('UCch-g_np5WDQ9ij7CYLrI5Q',), ('UCe4QsdtZez5As391tzqo-7w',), ('UC7hv6V49I7swZIRmXBFVwLw',), ('UCcC2GY6XydggENLpcHIvrBA',), ('UCYctOAjl8RwxBCK6NlrVT-A',), ('UCp9v5Q6s2d_z1veY0XlS0sw',), ('UCTvSwcc5QfrOvlwsRp7Jdkg',), ('UCRxJ9juyvkuuqYbrBo-XgDQ',), ('UCeUF6Dout6yAhjtjceN8nrQ',), ('UC8faCqAK0uZnM9hXI1Qk8xw',), ('UCQNGVe31gbZOy4d1gHapxhA',), ('UC27v4mmvgzj_FNR7wCpC-6w',), ('UCpgHjOxIX3hHHqjZmApSO4Q',), ('UCsM7-d9_ytHv7m10vjyKdtA',), ('UCbEtxe1mYGy-GEGaC8QLaBQ',), ('UC9fxyl7ooZ7ubFNJg6JbbPA',), ('UCKQKLYeuCqcQbLV4-1chMjQ',), ('UC4hs6L5YmSs6hUmajWCBciA',), ('UC1xOodWhUU4BOLuSkdxFFBg',), ('UCjhzCd-CwbB8S3JAbHtOppg',), ('UChasF4_C2X8KwAlYzwFZm-g',), ('UCHyDU7TOALJj1ITzlNGj1HA',), ('UCkmeteRExUxbtPVtLTy7brw',), ('UCvjjRRrjwYj3aKCW4zIyszw',), ('UCXXLmTUI4nSXE-gCTk4zCNw',), ('UC6CuXpTj5CxVRlau_uK-hMw',), ('UC7wy

In [ ]:
# Pour chaque chaîne, récupérer et insérer les métriques
for (id_chaine,) in chaines:
    metriques = getMetrics(id_chaine)
    if metriques:
        cur.execute("""
            INSERT INTO chaines_metriques (
                id_chaine, date_releve_chaine,
                nombre_vues_total, nombre_abonnes_total, nombre_videos_total
            )
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (id_chaine, date_releve_chaine) DO NOTHING
        """, (
            id_chaine,
            metriques['date_releve_chaine'],
            metriques['nombre_vues_total'],
            metriques['nombre_abonnes_total'],
            metriques['nombre_videos_total']
        ))

conn.commit()
cur.close()
conn.close()

# Calcul de la couverture

$$
\text{Couverture} = \frac{\text{Nombre de vidéos pertinentes collectées}}{\text{Nombre total de vidéos (le plus récent) de la chaîne pertinente}}
$$


In [23]:
def get_couverture(id_chaine,conn):
    
    cur = conn.cursor()

    # Récupérer le nombre de vidéos collectées
    cur.execute("""
        SELECT COUNT(*) FROM videos
        WHERE id_chaine = %s
    """, (id_chaine,))
    nb_collectees = cur.fetchone()[0]

    # Récupérer le nombre total de vidéos le plus récent
    cur.execute("""
        SELECT nombre_videos_total
        FROM chaines_metriques
        WHERE id_chaine = %s
        ORDER BY date_releve_chaine DESC
        LIMIT 1
    """, (id_chaine,))
    row = cur.fetchone()


    if not row:
        return None  # Pas de métrique pour cette chaîne

    nb_total = row[0]

    if nb_total == 0:
        return 0.0  # Évite division par zéro

    couverture = nb_collectees / nb_total
    return round(couverture, 3)


In [24]:
conn = psycopg.connect(
        dbname="mydatabase",
        user="postgres",
        password=os.getenv("POSTGRE_PASSWORD"),
        host="localhost",
        port="5432"
    )

couverture = get_couverture("UCxBJustR1tuXVy7tLivER2g",conn)
print("Couverture :", couverture)

conn.close()


Couverture : 0.016


In [27]:
def get_couverture_moyenne():
    conn = psycopg.connect(
        dbname="mydatabase",
        user="postgres",
        password=os.getenv("POSTGRE_PASSWORD"),
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    cur.execute("""
        SELECT id_chaine FROM chaines WHERE pertinente = TRUE
    """)
    chaines = cur.fetchall()

    total = 0.0
    count = 0

    for (id_chaine,) in tqdm(chaines):
        couverture = get_couverture(id_chaine,conn)
        if couverture is not None:
            total += couverture
            count += 1

    conn.close()

    if count == 0:
        return 0.0
    return round(total / count, 3)


In [28]:
get_couverture_moyenne()

100%|██████████| 280/280 [00:00<00:00, 380.83it/s]


0.081